In [29]:
import sys
sys.path.append("/home/sara/lib")

In [30]:
import pandas as pd
import os
import numpy as np
import libopf_py
import pandas as pd
import time
import random

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle

from sklearn.svm import SVC
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
#from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold # import KFold

from sklearn.model_selection import cross_validate

from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.model_selection import cross_val_score

In [31]:
df = pd.read_csv("csv_dataset/dataframe_7emocoes com numeros.csv", header=None, sep=';')
df = df.drop([0], axis=0)
df = df.drop(columns=[0])
df = shuffle(df)
print(df)

        1      2      3      4      5      6      7      8      9      10   \
16     85.0  209.0   88.0  258.0   96.0  307.0  101.0  355.0  111.0  403.0   
1910   70.0  241.0   78.0  291.0   89.0  338.0   99.0  385.0  112.0  432.0   
1742   75.0  215.0   80.0  264.0   88.0  311.0   93.0  359.0  105.0  406.0   
1533  101.0  228.0  104.0  279.0  111.0  329.0  118.0  379.0  128.0  428.0   
1566  102.0  217.0  105.0  268.0  112.0  316.0  118.0  366.0  127.0  415.0   
1626   82.0  229.0   85.0  279.0   93.0  327.0  101.0  374.0  112.0  422.0   
961   100.0  223.0  103.0  274.0  111.0  322.0  118.0  370.0  129.0  417.0   
1891   81.0  222.0   84.0  273.0   91.0  323.0   96.0  372.0  107.0  420.0   
1122   95.0  202.0   97.0  253.0  104.0  302.0  108.0  351.0  116.0  399.0   
591    67.0  215.0   73.0  266.0   84.0  316.0   94.0  364.0  108.0  410.0   
769    81.0  223.0   87.0  273.0   96.0  322.0  104.0  370.0  116.0  416.0   
1664   64.0  228.0   70.0  280.0   79.0  331.0   89.0  379.0  10

In [32]:
features = df.iloc[0:1925,0:136]
print(type(features))
features = features.to_numpy()
features = features.astype('float64')

print(features)
print(len(features))

<class 'pandas.core.frame.DataFrame'>
[[ 85. 209.  88. ... 413. 240. 410.]
 [ 70. 241.  78. ... 414. 246. 413.]
 [ 75. 215.  80. ... 409. 229. 405.]
 ...
 [103. 221. 106. ... 431. 255. 429.]
 [ 78. 237.  85. ... 421. 241. 421.]
 [ 91. 208.  93. ... 424. 239. 421.]]
1924


In [33]:
labels = df.iloc[0:,136]
labels = labels.to_numpy()
labels = labels.astype('float64')

print(labels)
print(len(labels))

[ 0. 22. 22. ...  0.  9.  0.]
1924


In [ ]:
n_samples = len(features)
print(n_samples)

In [34]:
train_feat,test_feat,train_labels,test_labels = train_test_split(features,labels, test_size=0.20,random_state=42 )

print ("Training set:", train_feat.shape[0], "samples")
print ("Test set:", test_feat.shape[0], "samples")

Training set: 1539 samples
Test set: 385 samples


In [35]:
# adaptações ao OPF.
def opf(train_labels1, train_feat1, test_labels1, test_feat1):
        
    # OPF only supports 32 bits labels at the moment
    label_train_32 = train_labels1.astype(np.int32)
    label_test_32 = test_labels1.astype(np.int32)

    O = libopf_py.OPF()

    O.fit(train_feat1, label_train_32)

    predicted = O.predict(test_feat1)
    name = "OPF"
    acc = accuracy_score(label_test_32, predicted)
    
    return acc  

In [42]:
C=1
models = (SVC(kernel='linear', C=C, gamma='auto'),
          SVC(kernel='rbf', gamma=0.5, C=C),
          SVC(kernel='poly', degree=3, C=C, gamma='auto'),
          KNeighborsClassifier(3),
          DecisionTreeClassifier(max_depth=8),
          RandomForestClassifier(max_depth=8, n_estimators=10, max_features=1),          
          AdaBoostClassifier(),
          GaussianNB(),
         )

names = ["SVC with linear kernel","SVC with RBF kernel","SVC with polynomial (degree 3) kernel",
         "K Nearest Neighbors","Decision Tree", "Random Forest", "AdaBoost","Naive Bayes", "OPF"]


In [66]:
def kfold(X, y):
    #X_train = []
    #X_test = []
    #y_train = []
    #y_test = []
    
    kf = KFold(n_splits = 8) # Define the split - number of folds 
    kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validatorprint(kf) KFold(n_splits=2, random_state=None, shuffle=False)

    matrix_acc = pd.DataFrame()
    fold = []
    medias = []
    i = 0
    
    for train_index, test_index in kf.split(X):
    
        train_feat, test_feat = X[train_index], X[test_index]
        train_labels, test_labels = y[train_index], y[test_index]
    
        for name, clf in zip(names, models):
            clf.fit(train_feat,train_labels) #train each model
            fold.append(clf.score(test_feat,test_labels)) #evaluate each model in the test set
        res_opf = opf(train_labels, train_feat, test_labels, test_feat)
        fold.append(res_opf)
        
        #print(fold)
        data = pd.DataFrame([fold])
        matrix_acc = matrix_acc.append(data)
        fold = []
    
    return matrix_acc
    
        #X_test.append(X[test_index])   # Uso isso se quiser debugar e saber os folds.
        #X_train.append(X[train_index])
        #y_test.append(y[test_index])
        #y_train.append(y[train_index])
        
    #return X_train, X_test, y_train, y_test

In [38]:
def names_emotions(labels_num):
    i = 0
    labels = []
    while (i < 1924):
        if labels_num[i] == 0.0:
            labels.append("neutro") 
        if labels_num[i] == 2.0:
            labels.append("alegria")
        if labels_num[i] == 9.0:
            labels.append("surpresa")
        if labels_num[i] == 12.0:
            labels.append("tristeza")
        if labels_num[i] == 13.0:
            labels.append("medo")
        if labels_num[i] == 21.0:
            labels.append("nojo")
        if labels_num[i] == 22.0:
            labels.append("raiva")
        i += 1
    return labels

In [39]:
matrix = kfold(features, labels)

[0.7842323651452282, 0.1950207468879668, 0.8049792531120332, 0.5311203319502075, 0.5269709543568465, 0.49377593360995853, 0.45643153526970953, 0.46887966804979253, 0.5518672199170125]
[0.7842323651452282, 0.15767634854771784, 0.7966804979253111, 0.5186721991701245, 0.5020746887966805, 0.42323651452282157, 0.4149377593360996, 0.4066390041493776, 0.5269709543568465]
[0.7593360995850622, 0.1991701244813278, 0.7800829875518672, 0.5435684647302904, 0.44813278008298757, 0.5311203319502075, 0.4730290456431535, 0.44813278008298757, 0.5560165975103735]
[0.7883817427385892, 0.18672199170124482, 0.8091286307053942, 0.5643153526970954, 0.49377593360995853, 0.47717842323651455, 0.46473029045643155, 0.4190871369294606, 0.5809128630705395]
[0.7875, 0.2125, 0.7958333333333333, 0.55, 0.5291666666666667, 0.5208333333333334, 0.48333333333333334, 0.4083333333333333, 0.5166666666666667]
[0.8, 0.17083333333333334, 0.8208333333333333, 0.5791666666666667, 0.4666666666666667, 0.5083333333333333, 0.454166666666

In [44]:
print ("      Classifiers: \t Accuracy (standard deviation)".expandtabs(37))
j = 0
for j in range(9):
    cl = matrix.iloc[0:,j] # separar todos os classificadores para depois fazer suas medias e desvios.
    #print(cl)
    #print("media", cl.mean())
    #print("desvio padrão", cl.std())
    print ("{:41} {:.3f} (+/- {:.3f}) ".format(names[j], cl.mean(), cl.std() * 2))

      Classifiers:                    Accuracy (standard deviation)
SVC with linear kernel                    0.786 (+/- 0.033) 
SVC with RBF kernel                       0.190 (+/- 0.037) 
SVC with polynomial (degree 3) kernel     0.806 (+/- 0.037) 
K Nearest Neighbors                       0.536 (+/- 0.058) 
Decision Tree                             0.498 (+/- 0.060) 
Random Forest                             0.502 (+/- 0.080) 
AdaBoost                                  0.466 (+/- 0.053) 
Naive Bayes                               0.433 (+/- 0.057) 
OPF                                       0.542 (+/- 0.062) 


# A partir daqui vou fazer o One hot encoding

In [ ]:
y_oh = pd.get_dummies(labels) # 
print(y_oh)
i = 0
emotion = []
for i in range(8):
    label = y_oh.iloc[0:,i]
    label = label.to_numpy()
    matrix = kfold(features, label)
    results(matrix)

      0.0   2.0   9.0   12.0  13.0  21.0  22.0
0        1     0     0     0     0     0     0
1        0     0     0     0     0     0     1
2        0     0     0     0     0     0     1
3        0     0     0     0     0     1     0
4        0     0     0     0     0     1     0
5        0     0     0     0     0     1     0
6        0     0     0     1     0     0     0
7        0     0     0     0     0     0     1
8        0     0     0     1     0     0     0
9        0     0     1     0     0     0     0
10       0     0     1     0     0     0     0
11       0     0     0     0     0     1     0
12       0     0     0     1     0     0     0
13       0     0     0     0     1     0     0
14       0     0     0     0     1     0     0
15       0     1     0     0     0     0     0
16       0     0     0     1     0     0     0
17       0     0     1     0     0     0     0
18       0     0     1     0     0     0     0
19       0     0     0     0     0     1     0
20       0   

In [65]:
def results(matrix1):
    print ("      Classifiers: \t Accuracy (standard deviation)".expandtabs(37))
    j = 0
    for j in range(9):
        cl = matrix1.iloc[0:,j] # separar todos os classificadores para depois fazer suas medias e desvios.
        #print(cl)
        #print("media", cl.mean())
        #print("desvio padrão", cl.std())
        print ("{:41} {:.3f} (+/- {:.3f}) ".format(names[j], cl.mean(), cl.std() * 2))